# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [3]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [4]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


In [8]:
data['FullName'] = data['FirstName']+" "+data['LastName']

data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity,FullName
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16,Rosa Andersen
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20,Myron Murray
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11,Susan Stevenson
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6,Tricia Vincent
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20,Scott Burch


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [28]:
data_gr = data.groupby(['CustomerID','ProductName']).agg({'Quantity':'sum'})
data_gr.head()

Quantity
CustomerID ProductName                            
33         Apricots - Dried                      1
           Assorted Desserts                     1
           Bandage - Flexible Neon               1
           Bar Mix - Pina Colada, 355 Ml         1
           Beans - Kidney, Canned                1

In [15]:
cust_matr = data.pivot_table(index = 'ProductName', columns = 'CustomerID', values = 'Quantity', aggfunc = 'sum', fill_value = 0)
cust_matr

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
ProductName,,,,,,,,,,,,,,,,,,,,,
Anchovy Paste - 56 G Tube,0,0,0,0,0,0,0,1,0,0,...,0,25,0,0,0,0,0,0,0,0
"Appetizer - Mini Egg Roll, Shrimp",0,0,0,0,0,0,0,0,0,0,...,25,25,0,0,0,0,0,0,0,0
Appetizer - Mushroom Tart,0,0,0,0,0,0,0,1,0,0,...,25,0,0,0,0,0,0,0,25,0
Appetizer - Sausage Rolls,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,25,25,25,0,25,0
Apricots - Dried,1,0,0,0,1,0,0,0,0,0,...,0,25,0,0,0,0,0,0,0,0
Apricots - Halves,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,25,50,25,0,25,25
Apricots Fresh,0,0,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,25
Arizona - Green Tea,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,25,0,0,0,0
Artichokes - Jerusalem,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [18]:
distances = pd.DataFrame(1/(1 + squareform(pdist(cust_matr.T, 'euclidean'))), 
                         index=cust_matr.columns, columns=cust_matr.columns)

distances.columns

Int64Index([   33,   200,   264,   356,   412,   464,   477,   639,   649,
              669,
            ...
            97697, 97753, 97769, 97793, 97900, 97928, 98069, 98159, 98185,
            98200],
           dtype='int64', name='CustomerID', length=1000)

## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [63]:
similarities = distances[97753].sort_values(ascending=False)[1:6]
similarities

CustomerID
28875    0.004805
18796    0.004795
21095    0.004791
35877    0.004781
16809    0.004777
Name: 97753, dtype: float64

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [66]:
cust_ids = list(similarities.index)
cust_ids

[28875, 18796, 21095, 35877, 16809]

In [67]:
data_five = data_gr.loc[cust_ids]
data_five.head()

Quantity
CustomerID ProductName                      
16809      Apricots - Dried                5
           Bandage - Fexible 1x3           5
           Bandage - Flexible Neon         5
           Barramundi                      5
           Beans - Kidney, Red Dry         5

## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [39]:
#cust_select['total'] = cust_select[264]+cust_select[3535]+cust_select[3317]+cust_select[2503]+cust_select[3305]

#cust_select['total']

In [54]:
ranked = data_five.pivot_table(index = 'ProductName', values = 'Quantity', aggfunc = 'sum').sort_values('Quantity', ascending = False)
ranked.head()

,Quantity
ProductName,
Ecolab - Lime - A - Way 4/4 L,34
Cocoa Butter,30
Vinegar - Tarragon,29
Muffin Batt - Choc Chk,28
Papayas,26


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [57]:
df_merged = ranked.merge(cust_matr, on = 'ProductName')

In [77]:
filtered = df_merged[df_merged[97753] == 0]

filtered = filtered.reset_index()
prods = list(filtered.ProductName[:5])
prods

['Pickerel - Fillets',
 'Blueberries',
 'Baking Powder',
 'Pepper - Paprika, Hungarian',
 'Ecolab - Mikroklene 4/4 L']

## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [91]:
recs = {}

ids = list(data.CustomerID)

for i in ids:
    similarities = distances[i].sort_values(ascending=False)[1:6]
    cust_ids = list(similarities.index)
    data_five = data_gr.loc[cust_ids]
    ranked = data_five.pivot_table(index = 'ProductName', values = 'Quantity', aggfunc = 'sum').sort_values('Quantity', ascending = False)
    df_merged = ranked.merge(cust_matr, on = 'ProductName')
    filtered = df_merged[df_merged[i] == 0]
    filtered = filtered.reset_index()
    prods = list(filtered.ProductName[:5])
    recs.update( {i : prods} )

In [79]:
recs

{61288: ['Jagermeister',
  'Chicken - Soup Base',
  'Lime Cordial - Roses',
  'Macaroons - Two Bite Choc',
  'Flavouring - Orange'],
 77352: ['Wine - Valpolicella Masi',
  'Tahini Paste',
  'Tuna - Salad Premix',
  'Chicken - Soup Base',
  'Isomalt'],
 40094: ['Bread - Italian Corn Meal Poly',
  'Puree - Mocha',
  'Tuna - Salad Premix',
  'Sherry - Dry',
  'Beer - Sleemans Cream Ale'],
 23548: ['Squid - Tubes / Tenticles 10/20',
  'Sprouts - Baby Pea Tendrils',
  'Flavouring - Orange',
  'Bread - Raisin Walnut Oval',
  'Banana Turning'],
 78981: ['Vanilla Beans',
  'Lettuce - Frisee',
  'Yogurt - Blueberry, 175 Gr',
  'Cinnamon Buns Sticky',
  'Pop Shoppe Cream Soda'],
 83106: ['Cocoa Butter',
  'Cheese - Boursin, Garlic / Herbs',
  'Garlic - Peeled',
  'Cheese - Mix',
  'Sauce - Hollandaise'],
 11253: ['Juice - Lime',
  'Tomatoes Tear Drop',
  'Pomello',
  'Potatoes - Instant, Mashed',
  'Table Cloth - 53x69 Colour'],
 35107: ['Bread - French Baquette',
  'Bandage - Flexible Neon',
  

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [82]:
df_recs = pd.DataFrame.from_dict(recs, orient='index')
df_recs = df_recs.reset_index()
df_recs = df_recs.rename(columns={"index": "Customer ID", 0: "Product 1", 1: "Product 2", 2: "Product 3", 3: "Product 4", 4: "Product 5"})
df_recs

,Customer ID,Product 1,Product 2,Product 3,Product 4,Product 5
0,61288,Jagermeister,Chicken - Soup Base,Lime Cordial - Roses,Macaroons - Two Bite Choc,Flavouring - Orange
1,77352,Wine - Valpolicella Masi,Tahini Paste,Tuna - Salad Premix,Chicken - Soup Base,Isomalt
2,40094,Bread - Italian Corn Meal Poly,Puree - Mocha,Tuna - Salad Premix,Sherry - Dry,Beer - Sleemans Cream Ale
3,23548,Squid - Tubes / Tenticles 10/20,Sprouts - Baby Pea Tendrils,Flavouring - Orange,Bread - Raisin Walnut Oval,Banana Turning
4,78981,Vanilla Beans,Lettuce - Frisee,"Yogurt - Blueberry, 175 Gr",Cinnamon Buns Sticky,Pop Shoppe Cream Soda
5,83106,Cocoa Butter,"Cheese - Boursin, Garlic / Herbs",Garlic - Peeled,Cheese - Mix,Sauce - Hollandaise
6,11253,Juice - Lime,Tomatoes Tear Drop,Pomello,"Potatoes - Instant, Mashed",Table Cloth - 53x69 Colour
7,35107,Bread - French Baquette,Bandage - Flexible Neon,"Lentils - Red, Dry",Thermometer Digital,Wine - Hardys Bankside Shiraz
8,15088,Soup Knorr Chili With Beans,Soupfoamcont12oz 112con,Veal - Sweetbread,Cocktail Napkin Blue,Bread - Raisin Walnut Oval
9,26031,Cheese - Mozzarella,Bananas,Quiche Assorted,"Lemonade - Natural, 591 Ml",Coffee - Hazelnut Cream


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [83]:
distances_city = pd.DataFrame(1/(1 + squareform(pdist(cust_matr.T, 'cityblock'))), 
                         index=cust_matr.columns, columns=cust_matr.columns)

In [88]:
recs_city = {}

ids = list(data.CustomerID)

for i in ids:
    similarities = distances_city[i].sort_values(ascending=False)[1:6]
    cust_ids = list(similarities.index)
    data_five = data_gr.loc[cust_ids]
    ranked = data_five.pivot_table(index = 'ProductName', values = 'Quantity', aggfunc = 'sum').sort_values('Quantity', ascending = False)
    df_merged = ranked.merge(cust_matr, on = 'ProductName')
    filtered = df_merged[df_merged[i] == 0]
    filtered = filtered.reset_index()
    prods = list(filtered.ProductName[:5])
    recs_city.update( {i : prods} )

In [89]:
df_recs_city = pd.DataFrame.from_dict(recs_city, orient='index')
df_recs_city = df_recs_city.reset_index()
df_recs_city = df_recs_city.rename(columns={"index": "Customer ID", 0: "Product 1", 1: "Product 2", 2: "Product 3", 3: "Product 4", 4: "Product 5"})
df_recs_city

,Customer ID,Product 1,Product 2,Product 3,Product 4,Product 5
0,61288,Soup - Campbells Bean Medley,Butter - Unsalted,"Oranges - Navel, 72",Wine - Ej Gallo Sierra Valley,Bay Leaf
1,77352,Tia Maria,Juice - Lime,"Sole - Dover, Whole, Fresh",Veal - Sweetbread,Foam Dinner Plate
2,40094,Knife Plastic - White,Butter - Unsalted,Wine - Ej Gallo Sierra Valley,Beef Wellington,Soup - Campbells Bean Medley
3,23548,Beef - Texas Style Burger,"Wine - White, Colubia Cresh",Squid - Tubes / Tenticles 10/20,"Cheese - Boursin, Garlic / Herbs",Wine - Chardonnay South
4,78981,Veal - Sweetbread,Knife Plastic - White,Muffin - Carrot Individual Wrap,Quiche Assorted,"Salsify, Organic"
5,83106,Tia Maria,Beef - Montreal Smoked Brisket,Wine - Blue Nun Qualitatswein,Table Cloth 81x81 White,Pomello
6,11253,Juice - Lime,Tomatoes Tear Drop,"Pepper - Paprika, Hungarian",Butter - Unsalted,Skirt - 29 Foot
7,35107,Pork - Kidney,Bread Crumbs - Panko,Lime Cordial - Roses,Sage - Ground,"Cheese - Boursin, Garlic / Herbs"
8,15088,Butter - Unsalted,Knife Plastic - White,"Pepper - Black, Whole",Soup - Campbells Bean Medley,Cod - Black Whole Fillet
9,26031,Wine - Toasted Head,"Wine - Magnotta, Merlot Sr Vqa",Chinese Foods - Chicken,Wine - Blue Nun Qualitatswein,Cake - Cake Sheet Macaroon


In [ ]:
#the recommendations are completely different with euclidean and cityblock distance measures